# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** B

**Names:**

* Vincenzo Bazzucchi
* Amaury Combes
* Alexis Montavon

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [17]:
import networkx as nx
import numpy as np
from random import choice
from collections import Counter

In [18]:
def directed_graph_from_tsv(pathname):
    file = open(pathname, 'r');
    G = nx.DiGraph()
    for line in file:
        content = line.split('\t')
        u = int(content[0])
        adjacent_nodes = map(lambda v: int(v), content[1].split())
        for v in adjacent_nodes:
            G.add_edge(u, v)
    file.close()
    return G

In [28]:
# Question for these two functions: is having a maximum number of hopes the right stop criteria?
def random_surfer(G, max_hops=1000):
    counter = Counter()
    u = choice(G.nodes())
    for i in range(max_hops):
        links = list(G[u].keys())
        if len(links) == 0: # Dangling node found!
            break
        v = choice(links)
        counter[v] += 1
        u = v
    n = sum(counter.values())
    return {page: counter[page] / n for page in counter.keys()}

In [29]:
components = directed_graph_from_tsv('../data/components.graph')
absorbing = directed_graph_from_tsv('../data/absorbing.graph')

In [30]:
N_SIMUL = 100
visited_nodes_p = 0
for i in range(N_SIMUL):
    visited_nodes_p += len(random_surfer(components))
print(visited_nodes_p / N_SIMUL, '% of nodes was visited')

4.0 % of nodes was visited


In [31]:
N_SIMUL = 100
visited_nodes_p = 0
for i in range(N_SIMUL):
    visited_nodes_p += len(random_surfer(absorbing))
print(visited_nodes_p / N_SIMUL, '% of nodes was visited')

1.56 % of nodes was visited


We observe that even if the num ber of maximal hops is quite large, the number of visited nodes stays small.
This behavior is probably caused by dangling nodes.

#### Exercise 2.13

In [35]:
def random_surfer_improved(G, max_hops=1000, damping_factor=0.15):
    counter = Counter()
    u = choice(G.nodes())
    for i in range(max_hops):
        links = list(G[u].keys())
        # if restart or dangling node start at random
        if np.random.binomial(1, damping_factor) or len(links) == 0:
            v = choice(G.nodes())
        else: # choose link at random
            v = choice(links)
        counter[v] += 1
        u = v
    n = sum(counter.values())
    return {page: counter[page] / n for page in counter.keys()}

In [33]:
N_SIMUL = 10000
visited_nodes_p = 0
for i in range(N_SIMUL):
    visited_nodes_p += len(random_surfer_improved(components))
print(visited_nodes_p / N_SIMUL, '% of nodes was visited')

7.9915 % of nodes was visited


In [34]:
N_SIMUL = 10000
visited_nodes_p = 0
for i in range(N_SIMUL):
    visited_nodes_p += len(random_surfer(components))
print(visited_nodes_p / N_SIMUL, '% of nodes was visited')

4.0 % of nodes was visited


In [10]:
#Do you think that the PageRank scores make intuitive sense?

---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [11]:
g = directed_graph_from_tsv('../data/wikipedia.graph')

In [38]:
def transition_matrix(graph):
    #H = np.zeros((len(graph), len(graph))) # I DO NOT UNDERSTAND WHY I GET INDEX ERROR
    H = []
    for u in graph.nodes():
        out_degree_u = nx.degree(g, u)
        line = []
        for v in graph.nodes():
            #H[u, v] = 1 / out_degree_u if graph.has_edge(u, v) else 0 #SEE ABOVE
            line.append(1 / out_degree_u if graph.has_edge(u, v) else 0)
        H.append(line)
    return np.matrix(H)

def GoogleMatrix(graph, theta):
    N = len(graph)
    H = transition_matrix(graph)
    w = np.reshape(np.array([1 if nx.degree(g, node) == 0 else 0 for node in graph]), (N, 1))
    onesT = np.ones((1, N))
    Hhat = H + (1 / N) * (w @ onesT)
    return theta * Hhat + (1-theta) * (onesT.T @ onesT) / N

def PageRank(google_matrix, pi_0, max_iter = np.inf):
    prev = np.zeros(pi_0.shape)
    curr = pi_0
    niter = 0
    # Power method iteration to find convergence.
    while not np.allclose(curr, prev) and niter < max_iter:
        prev = curr
        product = curr @ google_matrix
        curr = product / np.linalg.norm(product)
        niter += 1
    print("Converged in", niter, "steps")
    return np.ravel(curr)

In [39]:
GM = GoogleMatrix(g, 0.85)
r = PageRank(GM, np.array([1 / len(g) for el in g]))

Converged in 17 steps


matrix([[ 0.00933078,  0.00367   ,  0.01056846, ...,  0.00433566,
          0.00390726,  0.0059433 ]])

In [43]:
def geti(item):
    return item[1]
ranking = list(zip(range(len(r)), r)).sort(couple[1], reverse=True)

NameError: name 'couple' is not defined

---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*